### laptime & rank &timediff dataset + oracle features 

base: 9.DeepModels/laptime_rank_dataset.ipynb

Build a time series dataset across all the oval races, including laptime, rank. When this dataset aims to be used in forecasting, covariates of the racing status can not be included, such as track_status and lap_status. However, they can be used in oracle test to tell the upper bound of performance of the predictor.

Change to a new dataset format that following telemetry dataset.

raw:
+  [(eventid, carids: carno -> rowid, datalist)]

datalist := [datalist_entry] in shape of #car_number

datalist_entry := [[laptime, rank, track_status, lap_status]], in shape of #totallaps x #featureCnt (padded by nan)

gluonts:

+ Rank2LaptimeDifference

rank prediction by elapsed time difference in a race

Since use laptime2rank is not very successful, and predicting rank directly is not as well, to combine the real value regression into rank prediction task might be a solution.

And, differences of the elapsed time among the cars takes correlations into input, that brings more chances to success.

time_behind_leader is accurate when the car runs in the same lap as the leader does, otherwise, the time is truncted. To get accurate differences, using elapsed time is needed.


In [3]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import random
import mxnet as mx
from mxnet import gluon
import pickle
import json
from gluonts.dataset.common import ListDataset
from gluonts.dataset.util import to_pandas
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

## Load Data

In [4]:
import os
os.getcwd()

'/scratch/hpda/indycar/notebook/11.OracleRank'

In [5]:
#
# parameters
#
#year = '2017'
year = '2018'
#event = 'Toronto'
#https://www.racing-reference.info/season-stats/2018/O/#
events_totalmiles=[256,500,372,268,500,310]
events_laplen = [1.022,2.5,1.5,0.894,2.5,1.25]
events = ['Phoenix','Indy500','Texas','Iowa','Pocono','Gateway']
events_id={key:idx for idx, key in enumerate(events)}
#events = ['Indy500']
#events = ['Phoenix']

In [4]:
# make indy car completed_laps dataset
# car_number, completed_laps, rank, elapsed_time, rank_diff, elapsed_time_diff 
def make_cl_data(dataset):

    # pick up data with valid rank
    rankdata = dataset.rename_axis('MyIdx').sort_values(by=['elapsed_time','MyIdx'], ascending=True)
    rankdata = rankdata.drop_duplicates(subset=['car_number', 'completed_laps'], keep='first')

    # resort by car_number, lap
    uni_ds = rankdata.sort_values(by=['car_number', 'completed_laps', 'elapsed_time'], ascending=True)    
    #uni_ds = uni_ds.drop(["unique_id", "best_lap", "current_status", "track_status", "lap_status",
    #                  "laps_behind_leade","laps_behind_prec","overall_rank","pit_stop_count",
    #                  "last_pitted_lap","start_position","laps_led"], axis=1)
    
    uni_ds = uni_ds.drop(["unique_id", "best_lap", 
                      "laps_behind_leade","laps_behind_prec","overall_rank","pit_stop_count",
                      "last_pitted_lap","start_position","laps_led"], axis=1)
        
    carnumber = set(uni_ds['car_number'])
    print('cars:', carnumber)
    print('#cars=', len(carnumber))
   
    # faster solution , uni_ds already sorted by car_number and lap
    uni_ds['rank_diff'] = uni_ds['rank'].diff()
    mask = uni_ds.car_number != uni_ds.car_number.shift(1)
    uni_ds['rank_diff'][mask] = 0
    
    uni_ds['time_diff'] = uni_ds['elapsed_time'].diff()
    mask = uni_ds.car_number != uni_ds.car_number.shift(1)
    uni_ds['time_diff'][mask] = 0
    
    #df = uni_ds[['car_number','completed_laps','rank','elapsed_time','rank_diff','time_diff']]
    #df = uni_ds[['car_number','completed_laps','rank',
    #             'rank_diff','time_diff',"current_status", "track_status", "lap_status",'elapsed_time']]
    
    df = uni_ds[['car_number','completed_laps','time_diff','rank','track_status', 'lap_status','elapsed_time']]
    
    return df

def make_lapstatus_data(dataset):
    final_lap = max(dataset.completed_laps)
    total_laps = final_lap + 1

    # get records for the cars that finish the race
    completed_car_numbers= dataset[dataset.completed_laps == final_lap].car_number.values
    completed_car_count = len(completed_car_numbers)

    print('count of completed cars:', completed_car_count)
    print('completed cars:', completed_car_numbers)
    
    #pick up one of them
    onecar = dataset[dataset['car_number']==completed_car_numbers[0]]
    onecar = onecar.drop_duplicates(subset=['car_number', 'completed_laps'], keep='first')
    return onecar[['completed_laps','track_status']]
    

In [5]:
def load_data(event, year):
    inputfile = '../data/final/C_'+ event +'-' + year + '-final.csv'
    outputprefix = year +'-' + event + '-'
    dataset = pd.read_csv(inputfile)
    #dataset.info(verbose=True)    
    
    final_lap = max(dataset.completed_laps)
    total_laps = final_lap + 1

    # get records for the cars that finish the race
    completed_car_numbers= dataset[dataset.completed_laps == final_lap].car_number.values
    completed_car_count = len(completed_car_numbers)

    print('count of completed cars:', completed_car_count)
    print('completed cars:', completed_car_numbers)

    #make a copy
    alldata = dataset.copy()
    dataset = dataset[dataset['car_number'].isin(completed_car_numbers)]
    rankdata = alldata.rename_axis('MyIdx').sort_values(by=['elapsed_time','MyIdx'], ascending=True)
    rankdata = rankdata.drop_duplicates(subset=['car_number', 'completed_laps'], keep='first')
    
    cldata = make_cl_data(dataset)
    flagdata = make_lapstatus_data(dataset)
    acldata = make_cl_data(alldata)

    return alldata, rankdata, acldata, flagdata

### overall view of laptime scatter plots



In [6]:
def get_cardata(curcarno, ycol='time_diff'):
    car = acldata[acldata['car_number']==curcarno]
    #print(car['time_diff'].describe())
    
    cols=['completed_laps','rank','car_number','lap_status','track_status',
    'pit_stop_count','current_status','start_position']
    colid={key:idx for idx, key in enumerate(cols)}

    cardata = rankdata[rankdata['car_number'] == curcarno]

    carstatus = [[row[0], row[1],row[2],row[3],row[4],row[5],row[6],row[7]] for row in cardata[
        ['completed_laps','rank','car_number','lap_status','track_status',
        'pit_stop_count','current_status','start_position']].values]
    
    x = car['completed_laps'][1:].values
    y = car[ycol][1:].values

    pits=[]
    yellowflags=[]
    lastflag = 'x'
    for row in carstatus:
        lap = int(row[colid['completed_laps']])

        if row[colid['lap_status']]=='P':
            pits.append(lap)

        if row[colid['track_status']]=='Y':
            if lastflag != 'Y':       
                #start
                yellowflags.append(lap)
        else:
            if lastflag == 'Y':       
                #end
                yellowflags.append(lap)        
        lastflag = row[colid['track_status']]

    #pit lap
    pits = np.array(pits)
    #start, end lap
    #
    yellowflags = np.array(yellowflags)
    if (yellowflags.shape[0] % 2)==1:
        print('crash?:carno=', curcarno)
        yellowflags = []
    else:
        yellowflags = np.array(yellowflags).reshape((-1,2))    
    
    return car, x, y, pits, yellowflags

### bulid the dataset

In [12]:
def get_lapdata(acldata):
    """
    input:
        acldata['car_number','completed_laps','time_diff','rank','track_status', 'lap_status','elapsed_time']
    
        timediff: [car_number, completed_laps] -> elapsed time diff to leader
    output:
        lapdata = acldata[['car_number','completed_laps',
                           'time_diff','rank','track_status', 'lap_status','time_behind']].to_numpy()
    """
    COL_COMPLETED_LAPS = 1
    COL_ELAPSED_TIME = 6
    
    maxlap = np.max(acldata['completed_laps'].values)
    #'car_number','completed_laps','time_diff','rank','track_status', 'lap_status','time_behind'
    time_behind = []
    
    for lap in range(1, maxlap+1):
        this_lap = acldata[acldata['completed_laps']==lap][
            ['car_number','completed_laps','time_diff','rank','track_status', 'lap_status','elapsed_time']].values
        
        min_elapsed_time = np.nanmin(this_lap[:,COL_ELAPSED_TIME].astype(np.float))
        print(f'lap:{lap}, min_elapsed_time:{min_elapsed_time}')
        
        for row in this_lap:
            car_number = int(row[0])
            time_diff = row[2]
            rank = row[3]
            track_status = row[4]
            lap_status = row[5]
            
            timebehind = float(row[COL_ELAPSED_TIME]) - min_elapsed_time
            #
            time_behind.append([car_number, lap, time_diff,rank,track_status, lap_status,
                                timebehind])
    
    #return
    lapdata = np.array(time_behind)
    return lapdata

def get_laptime_dataset(stagedata):
    """
    input: (alldata, rankdata, acldata, flagdata)
    output: laptime & rank data
    
    [(
    eventid,
    carids : rowid -> carno,
    datalist: #car_number x features x #totallaps (padded by Nan)
        entry: [[laptime, rank, track_status, lap_status]]
    )]
    """
    laptime_data = []
    for event in stagedata.keys():
        
        laptime_rec = []
        eventid = events_id[event]
        
        alldata, rankdata, acldata, flagdata = stagedata[event]
        carlist = set(acldata['car_number'])
        laplist = set(acldata['completed_laps'])
        totalcars = len(carlist)
        totallaps = len(laplist)

        #carnumber -> carid
        carids={key:idx for idx, key in enumerate(carlist)}
        decode_carids={idx:key for idx, key in enumerate(carlist)}
        
        #array: car_number x lap
        #laptime = np.zeros((totalcars, totallaps-1))
        #rank = np.zeros((totalcars, totallaps-1))
        laptime = np.empty((totalcars, totallaps-1))
        rank = np.empty((totalcars, totallaps-1))
        laptime[:] = np.NaN
        rank[:] = np.NaN
        
        # features: laptime, rank, track_status, lap_status, timediff
        LAPTIME = 0
        RANK = 1
        TRACK_STATUS = 2
        LAP_STATUS = 3
        TIME_BEHIND = 4
        featureCnt = 5
        datalist = np.empty((totalcars, featureCnt, totallaps-1))
        datalist[:] = np.NaN
        
        #lapdata = acldata[['car_number','completed_laps',
        #                   'time_diff','rank','track_status', 'lap_status','elapsed_time']].to_numpy()
        
        #'car_number','completed_laps','time_diff','rank','track_status', 'lap_status','time_behind'
        lapdata = get_lapdata(acldata)
        
        
        for row in lapdata:
            #completed_laps
            if int(row[1]) == 0:
                continue
                
            #add to data array
            car_number = carids[int(row[0])]
            completed_laps = int(row[1])-1
            time_diff = float(row[2])
            rank = int(row[3])
            track_status = 1 if row[4]=='Y' else 0
            lap_status = 1 if row[5]=='P' else 0
            time_behind = float(row[6])
            
            datalist[car_number, LAPTIME, completed_laps] = time_diff
            datalist[car_number, RANK, completed_laps] = rank
            datalist[car_number, TRACK_STATUS, completed_laps] = track_status
            datalist[car_number, LAP_STATUS, completed_laps] = lap_status
            datalist[car_number, TIME_BEHIND, completed_laps] = time_behind

        #add one record
        laptime_data.append([eventid, decode_carids, datalist])
        # push this event into stage dataframe
        print('event=%s, records=%s'%(event, datalist.shape))
        
    
    return laptime_data

### load data

In [13]:
stagedata = {}
global_carids = {}
traindata = None
cur_carid = 0
for event in events:
    #alldata, rankdata, acldata, flagdata
    stagedata[event] = load_data(event, year)
    
    alldata, rankdata, acldata, flagdata = stagedata[event]
    carlist = set(acldata['car_number'])
    laplist = set(acldata['completed_laps'])
    print('%s: carno=%d, lapnum=%d'%(event, len(carlist), len(laplist)))

    #build the carid map
    for car in carlist:
        if car not in global_carids:
            global_carids[car] = cur_carid
            cur_carid += 1
    

count of completed cars: 11
completed cars: [ 1  6 27  9 28  5 20 14 15 22 30]
cars: {1, 5, 6, 9, 14, 15, 20, 22, 27, 28, 30}
#cars= 11


/scratch/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/scratch/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


count of completed cars: 11
completed cars: [ 1  6 27  9 28  5 20 14 15 22 30]
cars: {1, 4, 5, 6, 9, 10, 12, 14, 15, 18, 19, 20, 21, 22, 23, 26, 27, 28, 30, 32, 59, 88, 98}
#cars= 23
Phoenix: carno=23, lapnum=251
count of completed cars: 18
completed cars: [12 20  9 27 28 22 29  1  6 15 66 98  4 88 25 60 64 23]
cars: {64, 1, 66, 98, 4, 6, 9, 12, 60, 15, 20, 22, 23, 88, 25, 27, 28, 29}
#cars= 18
count of completed cars: 18
completed cars: [12 20  9 27 28 22 29  1  6 15 66 98  4 88 25 60 64 23]
cars: {1, 3, 4, 6, 7, 9, 10, 12, 13, 14, 15, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 32, 33, 59, 60, 64, 66, 88, 98}
#cars= 33
Indy500: carno=33, lapnum=201
count of completed cars: 9
completed cars: [ 9 22 27  5 28 15 30 18 10]
cars: {5, 9, 10, 15, 18, 22, 27, 28, 30}
#cars= 9
count of completed cars: 9
completed cars: [ 9 22 27  5 28 15 30 18 10]
cars: {1, 3, 4, 5, 6, 7, 9, 10, 12, 14, 15, 18, 19, 20, 21, 22, 23, 25, 26, 27, 28, 30, 47, 55, 57, 59, 60, 68, 73, 83, 88, 98}
#cars= 

In [14]:
laptime_data = get_laptime_dataset(stagedata)

lap:1, min_elapsed_time:21.3627
lap:2, min_elapsed_time:41.9897
lap:3, min_elapsed_time:62.7844
lap:4, min_elapsed_time:83.7068
lap:5, min_elapsed_time:104.6713
lap:6, min_elapsed_time:125.7053
lap:7, min_elapsed_time:146.8021
lap:8, min_elapsed_time:168.0942
lap:9, min_elapsed_time:189.4638
lap:10, min_elapsed_time:210.8339
lap:11, min_elapsed_time:232.236
lap:12, min_elapsed_time:253.6439
lap:13, min_elapsed_time:275.2336
lap:14, min_elapsed_time:297.4393
lap:15, min_elapsed_time:319.8041
lap:16, min_elapsed_time:342.0411
lap:17, min_elapsed_time:364.0351
lap:18, min_elapsed_time:385.7947
lap:19, min_elapsed_time:407.5462
lap:20, min_elapsed_time:429.7774
lap:21, min_elapsed_time:452.161
lap:22, min_elapsed_time:474.7218
lap:23, min_elapsed_time:497.1016
lap:24, min_elapsed_time:519.5567
lap:25, min_elapsed_time:541.9742
lap:26, min_elapsed_time:564.5793
lap:27, min_elapsed_time:587.1371
lap:28, min_elapsed_time:609.8118
lap:29, min_elapsed_time:632.5118
lap:30, min_elapsed_time:655.

event=Phoenix, records=(23, 5, 250)
lap:1, min_elapsed_time:41.9372
lap:2, min_elapsed_time:82.7118
lap:3, min_elapsed_time:123.5185
lap:4, min_elapsed_time:164.3588
lap:5, min_elapsed_time:205.2607
lap:6, min_elapsed_time:246.1499
lap:7, min_elapsed_time:287.0799
lap:8, min_elapsed_time:328.0156
lap:9, min_elapsed_time:369.1834
lap:10, min_elapsed_time:410.1523
lap:11, min_elapsed_time:451.4525
lap:12, min_elapsed_time:492.7575
lap:13, min_elapsed_time:534.0333
lap:14, min_elapsed_time:575.2708
lap:15, min_elapsed_time:616.4624
lap:16, min_elapsed_time:657.6129
lap:17, min_elapsed_time:698.7046
lap:18, min_elapsed_time:739.7912
lap:19, min_elapsed_time:780.7983
lap:20, min_elapsed_time:821.7476
lap:21, min_elapsed_time:862.8096
lap:22, min_elapsed_time:903.9989
lap:23, min_elapsed_time:945.1452
lap:24, min_elapsed_time:986.2716
lap:25, min_elapsed_time:1028.2411
lap:26, min_elapsed_time:1070.6739
lap:27, min_elapsed_time:1112.6169
lap:28, min_elapsed_time:1155.0238
lap:29, min_elapsed

lap:48, min_elapsed_time:1413.6965
lap:49, min_elapsed_time:1438.0822
lap:50, min_elapsed_time:1462.6443
lap:51, min_elapsed_time:1487.1412
lap:52, min_elapsed_time:1511.6164
lap:53, min_elapsed_time:1536.3856
lap:54, min_elapsed_time:1561.0923
lap:55, min_elapsed_time:1585.719
lap:56, min_elapsed_time:1610.4377
lap:57, min_elapsed_time:1635.297
lap:58, min_elapsed_time:1659.8438
lap:59, min_elapsed_time:1684.5336
lap:60, min_elapsed_time:1709.5126
lap:61, min_elapsed_time:1733.6337
lap:62, min_elapsed_time:1758.0843
lap:63, min_elapsed_time:1783.0939
lap:64, min_elapsed_time:1808.771
lap:65, min_elapsed_time:1833.1642
lap:66, min_elapsed_time:1858.0142
lap:67, min_elapsed_time:1882.9697
lap:68, min_elapsed_time:1913.7068
lap:69, min_elapsed_time:1938.533
lap:70, min_elapsed_time:1963.882
lap:71, min_elapsed_time:2000.4973
lap:72, min_elapsed_time:2047.3724
lap:73, min_elapsed_time:2071.4546
lap:74, min_elapsed_time:2095.3735
lap:75, min_elapsed_time:2119.6287
lap:76, min_elapsed_time:

lap:34, min_elapsed_time:667.2023
lap:35, min_elapsed_time:687.2779
lap:36, min_elapsed_time:707.207
lap:37, min_elapsed_time:727.1765
lap:38, min_elapsed_time:747.6015
lap:39, min_elapsed_time:767.3939999999999
lap:40, min_elapsed_time:787.1239999999999
lap:41, min_elapsed_time:807.0422
lap:42, min_elapsed_time:827.3429
lap:43, min_elapsed_time:847.5066
lap:44, min_elapsed_time:867.8892
lap:45, min_elapsed_time:887.7177
lap:46, min_elapsed_time:907.7719
lap:47, min_elapsed_time:928.1848
lap:48, min_elapsed_time:948.0961
lap:49, min_elapsed_time:968.1225
lap:50, min_elapsed_time:988.6352
lap:51, min_elapsed_time:1009.3704
lap:52, min_elapsed_time:1029.965
lap:53, min_elapsed_time:1051.4096
lap:54, min_elapsed_time:1072.2559
lap:55, min_elapsed_time:1093.0651
lap:56, min_elapsed_time:1113.7132
lap:57, min_elapsed_time:1134.02
lap:58, min_elapsed_time:1154.566
lap:59, min_elapsed_time:1175.5229
lap:60, min_elapsed_time:1196.547
lap:61, min_elapsed_time:1217.9728
lap:62, min_elapsed_time:

lap:284, min_elapsed_time:5975.8734
lap:285, min_elapsed_time:5995.7086
lap:286, min_elapsed_time:6015.6558
lap:287, min_elapsed_time:6036.2708
lap:288, min_elapsed_time:6056.8852
lap:289, min_elapsed_time:6077.2717
lap:290, min_elapsed_time:6097.9271
lap:291, min_elapsed_time:6118.4775
lap:292, min_elapsed_time:6138.9839
lap:293, min_elapsed_time:6159.5403
lap:294, min_elapsed_time:6181.8412
lap:295, min_elapsed_time:6216.8929
lap:296, min_elapsed_time:6259.1907
lap:297, min_elapsed_time:6303.5896
lap:298, min_elapsed_time:6353.8922
lap:299, min_elapsed_time:6405.357
lap:300, min_elapsed_time:6452.4666
event=Iowa, records=(22, 5, 300)
lap:1, min_elapsed_time:77.6277
lap:2, min_elapsed_time:182.1374
lap:3, min_elapsed_time:287.21
lap:4, min_elapsed_time:386.2278
lap:5, min_elapsed_time:477.4765
lap:6, min_elapsed_time:569.5639
lap:7, min_elapsed_time:615.2506
lap:8, min_elapsed_time:777.5886
lap:9, min_elapsed_time:947.1411
lap:10, min_elapsed_time:1057.502
lap:11, min_elapsed_time:116

lap:33, min_elapsed_time:1069.9472
lap:34, min_elapsed_time:1095.8487
lap:35, min_elapsed_time:1121.8185
lap:36, min_elapsed_time:1147.9482
lap:37, min_elapsed_time:1173.8102
lap:38, min_elapsed_time:1199.6706
lap:39, min_elapsed_time:1225.6212
lap:40, min_elapsed_time:1251.6131
lap:41, min_elapsed_time:1277.7327
lap:42, min_elapsed_time:1303.7533
lap:43, min_elapsed_time:1330.0615
lap:44, min_elapsed_time:1356.2028
lap:45, min_elapsed_time:1382.4273
lap:46, min_elapsed_time:1408.5839
lap:47, min_elapsed_time:1434.7908
lap:48, min_elapsed_time:1460.8174
lap:49, min_elapsed_time:1487.1647
lap:50, min_elapsed_time:1513.6129
lap:51, min_elapsed_time:1539.6393
lap:52, min_elapsed_time:1565.7409
lap:53, min_elapsed_time:1591.9083
lap:54, min_elapsed_time:1618.0414
lap:55, min_elapsed_time:1644.3939
lap:56, min_elapsed_time:1670.3496
lap:57, min_elapsed_time:1696.2503
lap:58, min_elapsed_time:1722.1997
lap:59, min_elapsed_time:1748.1958
lap:60, min_elapsed_time:1774.9077
lap:61, min_elapsed_

In [19]:
#check Car12 Indy500
laptime_data[1][2][7,:,100:110]

array([[43.0247, 43.6091, 42.9706, 42.3693, 41.8856, 41.004 , 40.7386,
        41.1659, 41.6323, 41.2931],
       [ 6.    ,  6.    ,  6.    ,  5.    ,  5.    ,  2.    ,  2.    ,
         1.    ,  1.    ,  1.    ],
       [ 0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,
         0.    ,  0.    ,  0.    ],
       [ 0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,
         0.    ,  0.    ,  0.    ],
       [ 9.6628, 11.6339, 13.0062, 13.4032, 13.2111,  2.6789,  1.8688,
         0.    ,  0.    ,  0.    ]])

In [20]:
#pocono crash
laptime_data[4][2][7,:,:50]

array([[ 92.5507, 101.063 , 104.9568, 100.7178,  91.1552,  80.7788,
         63.8658, 157.409 , 162.3938, 105.6318, 110.3061,  93.895 ,
         45.941 ,  43.3921,  43.4406, 448.2418,      nan,      nan,
             nan,      nan,      nan,      nan,      nan,      nan,
             nan,      nan,      nan,      nan,      nan,      nan,
             nan,      nan,      nan,      nan,      nan,      nan,
             nan,      nan,      nan,      nan,      nan,      nan,
             nan,      nan,      nan,      nan,      nan,      nan,
             nan,      nan],
       [ 14.    ,  14.    ,  14.    ,  16.    ,  16.    ,  16.    ,
         11.    ,  11.    ,  11.    ,  11.    ,   8.    ,   8.    ,
          8.    ,   8.    ,   8.    ,  16.    ,      nan,      nan,
             nan,      nan,      nan,      nan,      nan,      nan,
             nan,      nan,      nan,      nan,      nan,      nan,
             nan,      nan,      nan,      nan,      nan,      nan,
             nan,  

In [9]:
alldata, rankdata, acldata, flagdata = stagedata['Indy500']
acldata[acldata['car_number']==12].head(10)

,car_number,completed_laps,rank,elapsed_time,rank_diff,time_diff,current_status,track_status,lap_status
MyIdx,,,,,,,,,
2,12,0,3,0.2247,0.0,0.0000,Active,G,T
37,12,1,2,42.1485,-1.0,41.9238,Active,G,T
148,12,2,2,83.1845,0.0,41.0360,Active,G,T
231,12,3,3,124.5184,1.0,41.3339,Active,G,T
311,12,4,3,165.6102,0.0,41.0918,Active,G,T
393,12,5,3,206.4005,0.0,40.7903,Active,G,T
474,12,6,3,247.5158,0.0,41.1153,Active,G,T
558,12,7,3,288.8156,0.0,41.2998,Active,G,T
644,12,8,3,330.0010,0.0,41.1854,Active,G,T


In [11]:
lapdata = acldata[['car_number','completed_laps','time_diff','rank','track_status']].to_numpy()
lapdata[:10]

array([[1, 0, 0.0, 4, 'G'],
       [1, 1, 42.3679, 4, 'G'],
       [1, 2, 41.0984, 4, 'G'],
       [1, 3, 41.094300000000004, 4, 'G'],
       [1, 4, 41.11800000000001, 4, 'G'],
       [1, 5, 41.07169999999999, 4, 'G'],
       [1, 6, 41.18700000000001, 4, 'G'],
       [1, 7, 41.26490000000001, 4, 'G'],
       [1, 8, 41.34119999999996, 4, 'G'],
       [1, 9, 41.36439999999999, 4, 'G']], dtype=object)

In [21]:
import pickle
#stintdf.to_csv('laptime-%s.csv'%year)
with open(f'laptime_rank_timediff-oracle-{year}.pickle', 'wb') as f:
    #pack [global_carids, laptime_data]
    savedata = [global_carids, laptime_data]
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(savedata, f, pickle.HIGHEST_PROTOCOL)

### convert to gluonts dataset

In [6]:
# start from here
import pickle
with open('laptime_rank_timediff-oracle-%s.pickle'%year, 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    global_carids, laptime_data = pickle.load(f, encoding='latin1') 

In [17]:

def nan_helper(y):
    """Helper to handle indices and logical indices of NaNs.

    Input:
        - y, 1d numpy array with possible NaNs
    Output:
        - nans, logical indices of NaNs
        - index, a function, with signature indices= index(logical_indices),
          to convert logical indices of NaNs to 'equivalent' indices
    Example:
        >>> # linear interpolation of NaNs
        >>> nans, x= nan_helper(y)
        >>> y[nans]= np.interp(x(nans), x(~nans), y[~nans])
    """

    return np.isnan(y), lambda z: z.nonzero()[0]

def test_flag(a, bitflag):
    return (a & bitflag) ==  bitflag

#
# remove NaN at the tail
# there should be no nans in the middle of the ts
COL_LAPTIME=0
COL_RANK=1
COL_TRACKSTATUS=2
COL_LAPSTATUS=3
COL_TIMEDIFF=4
MODE_ORACLE = 0
MODE_NOLAP = 1
MODE_NOTRACK = 2
MODE_TESTZERO = 4
MODE_TESTCURTRACK = 8
#MODE_STR={MODE_ORACLE:'oracle', MODE_NOLAP:'nolap',MODE_NOTRACK:'notrack',MODE_TEST:'test'}

def make_dataset(runs, prediction_length, freq, 
                       useeid = False,
                       run_ts=COL_LAPTIME, 
                       train_ratio = 0.8,
                       use_global_dict = True,
                       oracle_mode = MODE_ORACLE
                ):
    """
    split the ts to train and test part by the ratio
    
    oracle_mode: false to simulate prediction in real by 
        set the covariates of track and lap status as nan in the testset
            
    
    """    
    start = pd.Timestamp("01-01-2019", freq=freq)  # can be different for each time series

    train_set = []
    test_set = []
    
    #select run
    if runs>=0:
        _laptime_data = [laptime_data[runs].copy()]
    else:
        _laptime_data = laptime_data.copy()
    
   
    #_data: eventid, carids, datalist[carnumbers, features, lapnumber]->[laptime, rank, track, lap]]
    for _data in _laptime_data:
        _train = []
        _test = []
        
        #statistics on the ts length
        ts_len = [ _entry.shape[1] for _entry in _data[2]]
        train_len = int(np.max(ts_len) * train_ratio)
        
        print(f'====event:{events[_data[0]]}, train_len={train_len}, max_len={np.max(ts_len)}, min_len={np.min(ts_len)}')
                
        # process for each ts
        for rowid in range(_data[2].shape[0]):
            # rec[features, lapnumber] -> [laptime, rank, track_status, lap_status,timediff]]
            rec = _data[2][rowid].copy()
            
            #remove nan(only tails)
            nans, x= nan_helper(rec[run_ts,:])
            nan_count = np.sum(nans)             
            rec = rec[:, ~np.isnan(rec[run_ts,:])]
            
            # remove short ts
            totallen = rec.shape[1]
            if ( totallen < train_len + prediction_length):
                print(f'a short ts: carid={_data[1][rowid]}，len={totallen}')
                continue                
            
            if use_global_dict:
                carno = _data[1][rowid]
                carid = global_carids[_data[1][rowid]]
            else:
                #simulation dataset, todo, fix the carids as decoder
                carno = rowid
                carid = rowid
                
            
            if useeid:
                static_cat = [carid, _data[0]]    
            else:
                static_cat = [carid]    
                
            # selection of features
            if test_flag(oracle_mode, MODE_NOTRACK):                
                rec[COL_TRACKSTATUS, :] = 0
            if test_flag(oracle_mode, MODE_NOLAP):                
                rec[COL_LAPSTATUS, :] = 0
                
            # split and add to dataset record
            _train.append({'target': rec[run_ts,:train_len].astype(np.float32), 
                            'start': start, 
                            'feat_static_cat': static_cat,
                            'feat_dynamic_real': [rec[COL_TRACKSTATUS,:train_len],
                                   rec[COL_LAPSTATUS,:train_len]]
                          }
                          )
            
            # multiple test ts(rolling window as half of the prediction_length)
            test_rec_cnt = 0
            for endpos in range(totallen, train_len+prediction_length, 
                                -int(prediction_length/2)):
                
                track_rec = rec[COL_TRACKSTATUS, :endpos].copy()
                lap_rec = rec[COL_LAPSTATUS, :endpos].copy()
                
                # test mode
                if test_flag(oracle_mode, MODE_TESTCURTRACK):
                    # since nan does not work, use cur-val instead
                    track_rec[-prediction_length:] = track_rec[-prediction_length - 1]
                    #track_rec[-prediction_length:] = random.randint(0,1)
                    #lap_rec[-prediction_length:] = lap_rec[-prediction_length - 1]
                    lap_rec[-prediction_length:] = 0
                elif test_flag(oracle_mode, MODE_TESTZERO):
                    #set prediction part as nan
                    #track_rec[-prediction_length:] = np.nan
                    #lap_rec[-prediction_length:] = np.nan
                    track_rec[-prediction_length:] = 0
                    lap_rec[-prediction_length:] = 0                    
                
                _test.append({'target': rec[run_ts,:endpos].astype(np.float32), 
                            'start': start, 
                            'feat_static_cat': static_cat,
                            'feat_dynamic_real': [track_rec,lap_rec]
                            #'feat_dynamic_real': [rec[COL_TRACKSTATUS,:endpos],
                            #       rec[COL_LAPSTATUS,:endpos]] 
                             }
                          )   
                test_rec_cnt += 1
            
            #add one ts
            print(f'carno:{carno}, totallen:{totallen}, nancount:{nan_count}, test_reccnt:{test_rec_cnt}')

        train_set.extend(_train)
        test_set.extend(_test)

    print(f'train len:{len(train_set)}, test len:{len(test_set)}')
    
    train_ds = ListDataset(train_set, freq=freq)
    test_ds = ListDataset(test_set, freq=freq)    
    
    return train_ds, test_ds, train_set, test_set

def make_dataset_byevent(runs, prediction_length, freq, 
                       useeid = False,
                       run_ts=COL_LAPTIME, 
                       test_event = 'Indy500',
                       use_global_dict = True,
                       oracle_mode = MODE_ORACLE,
                       half_moving_win = True,
                       train_ratio=0.8
                ):
    """
    split the ts to train and test part by the ratio
    
    oracle_mode: false to simulate prediction in real by 
        set the covariates of track and lap status as nan in the testset
            
    
    """    
    start = pd.Timestamp("01-01-2019", freq=freq)  # can be different for each time series

    train_set = []
    test_set = []
    
    #select run
    if runs>=0:
        _laptime_data = [laptime_data[runs].copy()]
    else:
        _laptime_data = laptime_data.copy()
    
   
    #_data: eventid, carids, datalist[carnumbers, features, lapnumber]->[laptime, rank, track, lap]]
    for _data in _laptime_data:
        _train = []
        _test = []
        
        if events[_data[0]] == test_event:
            test_mode = True
        
        else:
            test_mode = False
            
            
        #statistics on the ts length
        ts_len = [ _entry.shape[1] for _entry in _data[2]]
        train_len = int(np.max(ts_len) * train_ratio)
        
        print(f'====event:{events[_data[0]]}, train_len={train_len}, max_len={np.max(ts_len)}, min_len={np.min(ts_len)}')
                
        # process for each ts
        for rowid in range(_data[2].shape[0]):
            # rec[features, lapnumber] -> [laptime, rank, track_status, lap_status,timediff]]
            rec = _data[2][rowid].copy()
            
            #remove nan(only tails)
            nans, x= nan_helper(rec[run_ts,:])
            nan_count = np.sum(nans)             
            rec = rec[:, ~np.isnan(rec[run_ts,:])]
            
            # remove short ts
            totallen = rec.shape[1]
            if ( totallen < train_len + prediction_length):
                print(f'a short ts: carid={_data[1][rowid]}，len={totallen}')
                continue                
            
            if use_global_dict:
                carno = _data[1][rowid]
                carid = global_carids[_data[1][rowid]]
            else:
                #simulation dataset, todo, fix the carids as decoder
                carno = rowid
                carid = rowid
                
            
            if useeid:
                static_cat = [carid, _data[0]]    
            else:
                static_cat = [carid]    
                
            # selection of features
            if test_flag(oracle_mode, MODE_NOTRACK):                
                rec[COL_TRACKSTATUS, :] = 0
            if test_flag(oracle_mode, MODE_NOLAP):                
                rec[COL_LAPSTATUS, :] = 0

            test_rec_cnt = 0
            if not test_mode:
                
                # all go to train set
                _train.append({'target': rec[run_ts,:].astype(np.float32), 
                                'start': start, 
                                'feat_static_cat': static_cat,
                                'feat_dynamic_real': [rec[COL_TRACKSTATUS,:],
                                       rec[COL_LAPSTATUS,:]]
                              }
                              )
            else:
                # reset train_len
                context_len = prediction_length*2
                if context_len < 10:
                    context_len = 10
                
                
                # multiple test ts(rolling window as half of the prediction_length)

                step = -int(prediction_length/2) if half_moving_win else -prediction_length
                for endpos in range(totallen, context_len+prediction_length, 
                                    step):

                    track_rec = rec[COL_TRACKSTATUS, :endpos].copy()
                    lap_rec = rec[COL_LAPSTATUS, :endpos].copy()

                    # test mode
                    if test_flag(oracle_mode, MODE_TESTCURTRACK):
                        # since nan does not work, use cur-val instead
                        track_rec[-prediction_length:] = track_rec[-prediction_length - 1]
                        #track_rec[-prediction_length:] = random.randint(0,1)
                        #lap_rec[-prediction_length:] = lap_rec[-prediction_length - 1]
                        lap_rec[-prediction_length:] = 0
                    elif test_flag(oracle_mode, MODE_TESTZERO):
                        #set prediction part as nan
                        #track_rec[-prediction_length:] = np.nan
                        #lap_rec[-prediction_length:] = np.nan
                        track_rec[-prediction_length:] = 0
                        lap_rec[-prediction_length:] = 0                    

                    _test.append({'target': rec[run_ts,:endpos].astype(np.float32), 
                                'start': start, 
                                'feat_static_cat': static_cat,
                                'feat_dynamic_real': [track_rec,lap_rec]
                                #'feat_dynamic_real': [rec[COL_TRACKSTATUS,:endpos],
                                #       rec[COL_LAPSTATUS,:endpos]] 
                                 }
                              )   
                    test_rec_cnt += 1
            
            #add one ts
            print(f'carno:{carno}, totallen:{totallen}, nancount:{nan_count}, test_reccnt:{test_rec_cnt}')

        train_set.extend(_train)
        test_set.extend(_test)

    print(f'train len:{len(train_set)}, test len:{len(test_set)}')
    
    train_ds = ListDataset(train_set, freq=freq)
    test_ds = ListDataset(test_set, freq=freq)    
    
    return train_ds, test_ds, train_set, test_set

def save_dataset(datafile,freq, prediction_length, cardinality, train_ds, test_ds):
    with open(datafile, 'wb') as f:
        #pack [global_carids, laptime_data]
        savedata = [freq, prediction_length, cardinality, train_ds, test_ds]
        #savedata = [freq, train_set, test_set]
        # Pickle the 'data' dictionary using the highest protocol available.
        pickle.dump(savedata, f, pickle.HIGHEST_PROTOCOL)

### bulid dataset

In [8]:
def make_gluontsdb(run_ts, dbname, train_ratio=0.8):
    #run on indy dataset
    train_ds, test_ds,_,_ = make_dataset(-1, prediction_length,freq,
                                         useeid=useeid, run_ts=run_ts,
                                        train_ratio=train_ratio)
    save_dataset(f'{dbname}-oracle-{ipstr}-all-all-f{freq}-t{prediction_length}-r{train_ratio}-gluonts-indy-2018.pickle'
                 , freq, prediction_length, cardinality,train_ds, test_ds)
    
    if useeid:
        #special for classic predictor only
        return
    
    train_ds, test_ds,_,_ = make_dataset(-1, prediction_length,freq, 
                                         oracle_mode=MODE_TESTZERO,useeid=useeid, 
                                         run_ts=run_ts,train_ratio=train_ratio)
    save_dataset(f'{dbname}-oracle-{ipstr}-all-zero-f{freq}-t{prediction_length}-r{train_ratio}-gluonts-indy-2018.pickle'
                 , freq, prediction_length, cardinality,train_ds, test_ds)
    
    train_ds, test_ds,_,_ = make_dataset(-1, prediction_length,freq,
                                         oracle_mode=MODE_TESTCURTRACK,useeid=useeid, 
                                         run_ts=run_ts,train_ratio=train_ratio)
    save_dataset(f'{dbname}-oracle-{ipstr}-all-curtrack-f{freq}-t{prediction_length}-r{train_ratio}-gluonts-indy-2018.pickle'
                 , freq, prediction_length, cardinality,train_ds, test_ds)    
    
    train_ds, test_ds,_,_ = make_dataset(-1, prediction_length,freq, 
                                         oracle_mode=MODE_NOTRACK,useeid=useeid, 
                                         run_ts=run_ts,train_ratio=train_ratio)
    save_dataset(f'{dbname}-oracle-{ipstr}-notrack-all-f{freq}-t{prediction_length}-r{train_ratio}-gluonts-indy-2018.pickle'
                 , freq, prediction_length, cardinality,train_ds, test_ds)
    train_ds, test_ds,_,_ = make_dataset(-1, prediction_length,freq, 
                                         oracle_mode=MODE_NOTRACK+MODE_TESTZERO,
                                         useeid=useeid, run_ts=run_ts,train_ratio=train_ratio)
    save_dataset(f'{dbname}-oracle-{ipstr}-notrack-zero-f{freq}-t{prediction_length}-r{train_ratio}-gluonts-indy-2018.pickle'
                 , freq, prediction_length, cardinality,train_ds, test_ds)
    
    
    train_ds, test_ds,_,_ = make_dataset(-1, prediction_length,freq, 
                                         oracle_mode=MODE_NOLAP,useeid=useeid, 
                                         run_ts=run_ts,train_ratio=train_ratio)
    save_dataset(f'{dbname}-oracle-{ipstr}-nolap-all-f{freq}-t{prediction_length}-r{train_ratio}-gluonts-indy-2018.pickle'
                 , freq, prediction_length, cardinality,train_ds, test_ds)
    train_ds, test_ds,_,_ = make_dataset(-1, prediction_length,freq, 
                                         oracle_mode=MODE_NOLAP+MODE_TESTZERO,useeid=useeid, 
                                         run_ts=run_ts,train_ratio=train_ratio)
    save_dataset(f'{dbname}-oracle-{ipstr}-nolap-zero-f{freq}-t{prediction_length}-r{train_ratio}-gluonts-indy-2018.pickle'
                 , freq, prediction_length, cardinality,train_ds, test_ds)
    train_ds, test_ds,_,_ = make_dataset(-1, prediction_length,freq, 
                                         oracle_mode=MODE_NOLAP+MODE_TESTCURTRACK,
                                         useeid=useeid, run_ts=run_ts,train_ratio=train_ratio)
    save_dataset(f'{dbname}-oracle-{ipstr}-nolap-curtrack-f{freq}-t{prediction_length}-r{train_ratio}-gluonts-indy-2018.pickle'
                 , freq, prediction_length, cardinality,train_ds, test_ds)    

In [18]:
def make_gluontsdb_byevent(run_ts, dbname, test_event='Indy500'):
    #run on indy dataset
    train_ds, test_ds,_,_ = make_dataset_byevent(-1, prediction_length,freq,
                                         useeid=useeid, run_ts=run_ts,
                                        test_event=test_event)
    save_dataset(f'{dbname}-oracle-{ipstr}-all-all-f{freq}-t{prediction_length}-r{test_event}-gluonts-indy-2018.pickle'
                 , freq, prediction_length, cardinality,train_ds, test_ds)
    
    if useeid:
        #special for classic predictor only
        return
    
    train_ds, test_ds,_,_ = make_dataset_byevent(-1, prediction_length,freq, 
                                         oracle_mode=MODE_TESTZERO,useeid=useeid, 
                                         run_ts=run_ts,test_event=test_event)
    save_dataset(f'{dbname}-oracle-{ipstr}-all-zero-f{freq}-t{prediction_length}-r{test_event}-gluonts-indy-2018.pickle'
                 , freq, prediction_length, cardinality,train_ds, test_ds)
    
    train_ds, test_ds,_,_ = make_dataset_byevent(-1, prediction_length,freq,
                                         oracle_mode=MODE_TESTCURTRACK,useeid=useeid, 
                                         run_ts=run_ts,test_event=test_event)
    save_dataset(f'{dbname}-oracle-{ipstr}-all-curtrack-f{freq}-t{prediction_length}-r{test_event}-gluonts-indy-2018.pickle'
                 , freq, prediction_length, cardinality,train_ds, test_ds)    
    
    train_ds, test_ds,_,_ = make_dataset_byevent(-1, prediction_length,freq, 
                                         oracle_mode=MODE_NOTRACK,useeid=useeid, 
                                         run_ts=run_ts,test_event=test_event)
    save_dataset(f'{dbname}-oracle-{ipstr}-notrack-all-f{freq}-t{prediction_length}-r{test_event}-gluonts-indy-2018.pickle'
                 , freq, prediction_length, cardinality,train_ds, test_ds)
    train_ds, test_ds,_,_ = make_dataset_byevent(-1, prediction_length,freq, 
                                         oracle_mode=MODE_NOTRACK+MODE_TESTZERO,
                                         useeid=useeid, run_ts=run_ts,test_event=test_event)
    save_dataset(f'{dbname}-oracle-{ipstr}-notrack-zero-f{freq}-t{prediction_length}-r{test_event}-gluonts-indy-2018.pickle'
                 , freq, prediction_length, cardinality,train_ds, test_ds)
    
    
    train_ds, test_ds,_,_ = make_dataset_byevent(-1, prediction_length,freq, 
                                         oracle_mode=MODE_NOLAP,useeid=useeid, 
                                         run_ts=run_ts,test_event=test_event)
    save_dataset(f'{dbname}-oracle-{ipstr}-nolap-all-f{freq}-t{prediction_length}-r{test_event}-gluonts-indy-2018.pickle'
                 , freq, prediction_length, cardinality,train_ds, test_ds)
    train_ds, test_ds,_,_ = make_dataset_byevent(-1, prediction_length,freq, 
                                         oracle_mode=MODE_NOLAP+MODE_TESTZERO,useeid=useeid, 
                                         run_ts=run_ts,test_event=test_event)
    save_dataset(f'{dbname}-oracle-{ipstr}-nolap-zero-f{freq}-t{prediction_length}-r{test_event}-gluonts-indy-2018.pickle'
                 , freq, prediction_length, cardinality,train_ds, test_ds)
    train_ds, test_ds,_,_ = make_dataset_byevent(-1, prediction_length,freq, 
                                         oracle_mode=MODE_NOLAP+MODE_TESTCURTRACK,
                                         useeid=useeid, run_ts=run_ts,test_event=test_event)
    save_dataset(f'{dbname}-oracle-{ipstr}-nolap-curtrack-f{freq}-t{prediction_length}-r{test_event}-gluonts-indy-2018.pickle'
                 , freq, prediction_length, cardinality,train_ds, test_ds) 

In [ ]:
### laptime dataset

freq = "1min"

useeid = False
interpolate = False
#ipstr = '-ip' if interpolate else '-noip'
ipstr = '%s-%s'%('ip' if interpolate else 'noip', 'eid' if useeid else 'noeid')
if useeid:
    cardinality = [len(global_carids), len(laptime_data)]
else:
    cardinality = [len(global_carids)]

plens=[2,5,10,20,30]
for prediction_length in plens:
    make_gluontsdb(COL_LAPTIME,'laptime')
    
#special
interpolate = True
useeid = False
ipstr = '%s-%s'%('ip' if interpolate else 'noip', 'eid' if useeid else 'noeid')
for prediction_length in plens:
    make_gluontsdb(COL_LAPTIME,'laptime')

useeid = True
interpolate = False
#ipstr = '-ip' if interpolate else '-noip'
ipstr = '%s-%s'%('ip' if interpolate else 'noip', 'eid' if useeid else 'noeid')
if useeid:
    cardinality = [len(global_carids), len(laptime_data)]
else:
    cardinality = [len(global_carids)]

for prediction_length in plens:
    make_gluontsdb(COL_LAPTIME,'laptime')
    

In [11]:
### rank dataset

freq = "1min"

useeid = False
interpolate = False
#ipstr = '-ip' if interpolate else '-noip'
ipstr = '%s-%s'%('ip' if interpolate else 'noip', 'eid' if useeid else 'noeid')
if useeid:
    cardinality = [len(global_carids), len(laptime_data)]
else:
    cardinality = [len(global_carids)]

plens=[2,5,10,20,30]
for prediction_length in plens:
    make_gluontsdb(COL_RANK,'rank')
    
#special
interpolate = True
useeid = False
ipstr = '%s-%s'%('ip' if interpolate else 'noip', 'eid' if useeid else 'noeid')
for prediction_length in plens:
    make_gluontsdb(COL_RANK,'rank')

useeid = True
interpolate = False
#ipstr = '-ip' if interpolate else '-noip'
ipstr = '%s-%s'%('ip' if interpolate else 'noip', 'eid' if useeid else 'noeid')
if useeid:
    cardinality = [len(global_carids), len(laptime_data)]
else:
    cardinality = [len(global_carids)]

for prediction_length in plens:
    make_gluontsdb(COL_RANK,'rank')

====event:Phoenix, train_len=200, max_len=250, min_len=250
carno:1, totallen:250, nancount:0, test_reccnt:48
carno:4, totallen:241, nancount:9, test_reccnt:39
carno:5, totallen:250, nancount:0, test_reccnt:48
carno:6, totallen:250, nancount:0, test_reccnt:48
carno:9, totallen:250, nancount:0, test_reccnt:48
carno:10, totallen:229, nancount:21, test_reccnt:27
a short ts: carid=12，len=154
carno:14, totallen:250, nancount:0, test_reccnt:48
carno:15, totallen:250, nancount:0, test_reccnt:48
carno:18, totallen:249, nancount:1, test_reccnt:47
a short ts: carid=19，len=40
carno:20, totallen:250, nancount:0, test_reccnt:48
carno:21, totallen:249, nancount:1, test_reccnt:47
carno:22, totallen:250, nancount:0, test_reccnt:48
carno:23, totallen:248, nancount:2, test_reccnt:46
carno:26, totallen:249, nancount:1, test_reccnt:47
carno:27, totallen:250, nancount:0, test_reccnt:48
carno:28, totallen:250, nancount:0, test_reccnt:48
carno:30, totallen:250, nancount:0, test_reccnt:48
a short ts: carid=32，

carno:18, totallen:297, nancount:3, test_reccnt:55
carno:19, totallen:291, nancount:9, test_reccnt:49
carno:20, totallen:298, nancount:2, test_reccnt:56
carno:21, totallen:300, nancount:0, test_reccnt:58
carno:22, totallen:299, nancount:1, test_reccnt:57
carno:23, totallen:295, nancount:5, test_reccnt:53
carno:26, totallen:279, nancount:21, test_reccnt:37
carno:27, totallen:298, nancount:2, test_reccnt:56
carno:28, totallen:283, nancount:17, test_reccnt:41
carno:30, totallen:300, nancount:0, test_reccnt:58
carno:59, totallen:294, nancount:6, test_reccnt:52
a short ts: carid=88，len=100
carno:98, totallen:293, nancount:7, test_reccnt:51
====event:Pocono, train_len=160, max_len=200, min_len=200
carno:1, totallen:199, nancount:1, test_reccnt:37
carno:4, totallen:197, nancount:3, test_reccnt:35
a short ts: carid=5，len=6
a short ts: carid=6，len=6
carno:9, totallen:200, nancount:0, test_reccnt:38
carno:10, totallen:197, nancount:3, test_reccnt:35
carno:12, totallen:200, nancount:0, test_reccn

carno:30, totallen:247, nancount:1, test_reccnt:47
carno:59, totallen:244, nancount:4, test_reccnt:44
carno:88, totallen:242, nancount:6, test_reccnt:42
carno:98, totallen:246, nancount:2, test_reccnt:46
train len:117, test len:5045
====event:Phoenix, train_len=200, max_len=250, min_len=250
carno:1, totallen:250, nancount:0, test_reccnt:48
carno:4, totallen:241, nancount:9, test_reccnt:39
carno:5, totallen:250, nancount:0, test_reccnt:48
carno:6, totallen:250, nancount:0, test_reccnt:48
carno:9, totallen:250, nancount:0, test_reccnt:48
carno:10, totallen:229, nancount:21, test_reccnt:27
a short ts: carid=12，len=154
carno:14, totallen:250, nancount:0, test_reccnt:48
carno:15, totallen:250, nancount:0, test_reccnt:48
carno:18, totallen:249, nancount:1, test_reccnt:47
a short ts: carid=19，len=40
carno:20, totallen:250, nancount:0, test_reccnt:48
carno:21, totallen:249, nancount:1, test_reccnt:47
carno:22, totallen:250, nancount:0, test_reccnt:48
carno:23, totallen:248, nancount:2, test_re

carno:9, totallen:296, nancount:4, test_reccnt:54
carno:10, totallen:295, nancount:5, test_reccnt:53
carno:12, totallen:299, nancount:1, test_reccnt:57
carno:14, totallen:292, nancount:8, test_reccnt:50
carno:15, totallen:299, nancount:1, test_reccnt:57
carno:18, totallen:297, nancount:3, test_reccnt:55
carno:19, totallen:291, nancount:9, test_reccnt:49
carno:20, totallen:298, nancount:2, test_reccnt:56
carno:21, totallen:300, nancount:0, test_reccnt:58
carno:22, totallen:299, nancount:1, test_reccnt:57
carno:23, totallen:295, nancount:5, test_reccnt:53
carno:26, totallen:279, nancount:21, test_reccnt:37
carno:27, totallen:298, nancount:2, test_reccnt:56
carno:28, totallen:283, nancount:17, test_reccnt:41
carno:30, totallen:300, nancount:0, test_reccnt:58
carno:59, totallen:294, nancount:6, test_reccnt:52
a short ts: carid=88，len=100
carno:98, totallen:293, nancount:7, test_reccnt:51
====event:Pocono, train_len=160, max_len=200, min_len=200
carno:1, totallen:199, nancount:1, test_reccn

carno:9, totallen:200, nancount:0, test_reccnt:38
carno:10, totallen:197, nancount:3, test_reccnt:35
carno:12, totallen:200, nancount:0, test_reccnt:38
a short ts: carid=14，len=16
carno:15, totallen:196, nancount:4, test_reccnt:34
carno:18, totallen:200, nancount:0, test_reccnt:38
a short ts: carid=19，len=6
carno:20, totallen:197, nancount:3, test_reccnt:35
a short ts: carid=21，len=17
carno:22, totallen:199, nancount:1, test_reccnt:37
carno:23, totallen:198, nancount:2, test_reccnt:36
carno:26, totallen:199, nancount:1, test_reccnt:37
carno:27, totallen:200, nancount:0, test_reccnt:38
a short ts: carid=28，len=6
a short ts: carid=30，len=6
carno:59, totallen:196, nancount:4, test_reccnt:34
carno:88, totallen:162, nancount:38, test_reccnt:0
carno:98, totallen:199, nancount:1, test_reccnt:37
====event:Gateway, train_len=198, max_len=248, min_len=248
carno:1, totallen:248, nancount:0, test_reccnt:48
carno:4, totallen:246, nancount:2, test_reccnt:46
carno:5, totallen:246, nancount:2, test_re

...

carno:12, totallen:248, nancount:0, test_reccnt:2
carno:14, totallen:246, nancount:2, test_reccnt:2
carno:15, totallen:247, nancount:1, test_reccnt:2
a short ts: carid=18，len=0
carno:19, totallen:247, nancount:1, test_reccnt:2
carno:20, totallen:247, nancount:1, test_reccnt:2
carno:21, totallen:248, nancount:0, test_reccnt:2
carno:22, totallen:248, nancount:0, test_reccnt:2
carno:23, totallen:235, nancount:13, test_reccnt:1
carno:26, totallen:248, nancount:0, test_reccnt:2
carno:27, totallen:248, nancount:0, test_reccnt:2
a short ts: carid=28，len=173
carno:30, totallen:247, nancount:1, test_reccnt:2
carno:59, totallen:244, nancount:4, test_reccnt:2
carno:88, totallen:242, nancount:6, test_reccnt:1
carno:98, totallen:246, nancount:2, test_reccnt:2
train len:112, test len:179
====event:Phoenix, train_len=200, max_len=250, min_len=250
carno:1, totallen:250, nancount:0, test_reccnt:2
carno:4, totallen:241, nancount:9, test_reccnt:1
carno:5, totallen:250, nancount:0, test_reccnt:2
carno:6, 

carno:18, totallen:200, nancount:0, test_reccnt:1
a short ts: carid=19，len=6
carno:20, totallen:197, nancount:3, test_reccnt:1
a short ts: carid=21，len=17
carno:22, totallen:199, nancount:1, test_reccnt:1
carno:23, totallen:198, nancount:2, test_reccnt:1
carno:26, totallen:199, nancount:1, test_reccnt:1
carno:27, totallen:200, nancount:0, test_reccnt:1
a short ts: carid=28，len=6
a short ts: carid=30，len=6
carno:59, totallen:196, nancount:4, test_reccnt:1
a short ts: carid=88，len=162
carno:98, totallen:199, nancount:1, test_reccnt:1
====event:Gateway, train_len=198, max_len=248, min_len=248
carno:1, totallen:248, nancount:0, test_reccnt:2
carno:4, totallen:246, nancount:2, test_reccnt:2
carno:5, totallen:246, nancount:2, test_reccnt:2
carno:9, totallen:248, nancount:0, test_reccnt:2
carno:10, totallen:248, nancount:0, test_reccnt:2
carno:12, totallen:248, nancount:0, test_reccnt:2
carno:14, totallen:246, nancount:2, test_reccnt:2
carno:15, totallen:247, nancount:1, test_reccnt:2
a short

carno:98, totallen:293, nancount:7, test_reccnt:51
====event:Pocono, train_len=160, max_len=200, min_len=200
carno:1, totallen:199, nancount:1, test_reccnt:37
carno:4, totallen:197, nancount:3, test_reccnt:35
a short ts: carid=5，len=6
a short ts: carid=6，len=6
carno:9, totallen:200, nancount:0, test_reccnt:38
carno:10, totallen:197, nancount:3, test_reccnt:35
carno:12, totallen:200, nancount:0, test_reccnt:38
a short ts: carid=14，len=16
carno:15, totallen:196, nancount:4, test_reccnt:34
carno:18, totallen:200, nancount:0, test_reccnt:38
a short ts: carid=19，len=6
carno:20, totallen:197, nancount:3, test_reccnt:35
a short ts: carid=21，len=17
carno:22, totallen:199, nancount:1, test_reccnt:37
carno:23, totallen:198, nancount:2, test_reccnt:36
carno:26, totallen:199, nancount:1, test_reccnt:37
carno:27, totallen:200, nancount:0, test_reccnt:38
a short ts: carid=28，len=6
a short ts: carid=30，len=6
carno:59, totallen:196, nancount:4, test_reccnt:34
carno:88, totallen:162, nancount:38, test_

====event:Phoenix, train_len=200, max_len=250, min_len=250
carno:1, totallen:250, nancount:0, test_reccnt:8
carno:4, totallen:241, nancount:9, test_reccnt:7
carno:5, totallen:250, nancount:0, test_reccnt:8
carno:6, totallen:250, nancount:0, test_reccnt:8
carno:9, totallen:250, nancount:0, test_reccnt:8
carno:10, totallen:229, nancount:21, test_reccnt:4
a short ts: carid=12，len=154
carno:14, totallen:250, nancount:0, test_reccnt:8
carno:15, totallen:250, nancount:0, test_reccnt:8
carno:18, totallen:249, nancount:1, test_reccnt:8
a short ts: carid=19，len=40
carno:20, totallen:250, nancount:0, test_reccnt:8
carno:21, totallen:249, nancount:1, test_reccnt:8
carno:22, totallen:250, nancount:0, test_reccnt:8
carno:23, totallen:248, nancount:2, test_reccnt:8
carno:26, totallen:249, nancount:1, test_reccnt:8
carno:27, totallen:250, nancount:0, test_reccnt:8
carno:28, totallen:250, nancount:0, test_reccnt:8
carno:30, totallen:250, nancount:0, test_reccnt:8
a short ts: carid=32，len=174
carno:59,

====event:Phoenix, train_len=200, max_len=250, min_len=250
carno:1, totallen:250, nancount:0, test_reccnt:2
carno:4, totallen:241, nancount:9, test_reccnt:1
carno:5, totallen:250, nancount:0, test_reccnt:2
carno:6, totallen:250, nancount:0, test_reccnt:2
carno:9, totallen:250, nancount:0, test_reccnt:2
a short ts: carid=10，len=229
a short ts: carid=12，len=154
carno:14, totallen:250, nancount:0, test_reccnt:2
carno:15, totallen:250, nancount:0, test_reccnt:2
carno:18, totallen:249, nancount:1, test_reccnt:2
a short ts: carid=19，len=40
carno:20, totallen:250, nancount:0, test_reccnt:2
carno:21, totallen:249, nancount:1, test_reccnt:2
carno:22, totallen:250, nancount:0, test_reccnt:2
carno:23, totallen:248, nancount:2, test_reccnt:2
carno:26, totallen:249, nancount:1, test_reccnt:2
carno:27, totallen:250, nancount:0, test_reccnt:2
carno:28, totallen:250, nancount:0, test_reccnt:2
carno:30, totallen:250, nancount:0, test_reccnt:2
a short ts: carid=32，len=174
carno:59, totallen:247, nancoun

In [24]:
### rank dataset

freq = "1min"

useeid = False
interpolate = False
#ipstr = '-ip' if interpolate else '-noip'
ipstr = '%s-%s'%('ip' if interpolate else 'noip', 'eid' if useeid else 'noeid')
if useeid:
    cardinality = [len(global_carids), len(laptime_data)]
else:
    cardinality = [len(global_carids)]

plens=[2,5,10,20,30]
for prediction_length in plens:
    make_gluontsdb(COL_TIMEDIFF,'timediff')
    
#special
interpolate = True
useeid = False
ipstr = '%s-%s'%('ip' if interpolate else 'noip', 'eid' if useeid else 'noeid')
for prediction_length in plens:
    make_gluontsdb(COL_TIMEDIFF,'timediff')

useeid = True
interpolate = False
#ipstr = '-ip' if interpolate else '-noip'
ipstr = '%s-%s'%('ip' if interpolate else 'noip', 'eid' if useeid else 'noeid')
if useeid:
    cardinality = [len(global_carids), len(laptime_data)]
else:
    cardinality = [len(global_carids)]

for prediction_length in plens:
    make_gluontsdb(COL_TIMEDIFF,'timediff')

====event:Phoenix, train_len=200, max_len=250, min_len=250
carno:1, totallen:250, nancount:0, test_reccnt:48
carno:4, totallen:241, nancount:9, test_reccnt:39
carno:5, totallen:250, nancount:0, test_reccnt:48
carno:6, totallen:250, nancount:0, test_reccnt:48
carno:9, totallen:250, nancount:0, test_reccnt:48
carno:10, totallen:229, nancount:21, test_reccnt:27
a short ts: carid=12，len=154
carno:14, totallen:250, nancount:0, test_reccnt:48
carno:15, totallen:250, nancount:0, test_reccnt:48
carno:18, totallen:249, nancount:1, test_reccnt:47
a short ts: carid=19，len=40
carno:20, totallen:250, nancount:0, test_reccnt:48
carno:21, totallen:249, nancount:1, test_reccnt:47
carno:22, totallen:250, nancount:0, test_reccnt:48
carno:23, totallen:248, nancount:2, test_reccnt:46
carno:26, totallen:249, nancount:1, test_reccnt:47
carno:27, totallen:250, nancount:0, test_reccnt:48
carno:28, totallen:250, nancount:0, test_reccnt:48
carno:30, totallen:250, nancount:0, test_reccnt:48
a short ts: carid=32，

carno:4, totallen:197, nancount:3, test_reccnt:35
a short ts: carid=5，len=6
a short ts: carid=6，len=6
carno:9, totallen:200, nancount:0, test_reccnt:38
carno:10, totallen:197, nancount:3, test_reccnt:35
carno:12, totallen:200, nancount:0, test_reccnt:38
a short ts: carid=14，len=16
carno:15, totallen:196, nancount:4, test_reccnt:34
carno:18, totallen:200, nancount:0, test_reccnt:38
a short ts: carid=19，len=6
carno:20, totallen:197, nancount:3, test_reccnt:35
a short ts: carid=21，len=17
carno:22, totallen:199, nancount:1, test_reccnt:37
carno:23, totallen:198, nancount:2, test_reccnt:36
carno:26, totallen:199, nancount:1, test_reccnt:37
carno:27, totallen:200, nancount:0, test_reccnt:38
a short ts: carid=28，len=6
a short ts: carid=30，len=6
carno:59, totallen:196, nancount:4, test_reccnt:34
carno:88, totallen:162, nancount:38, test_reccnt:0
carno:98, totallen:199, nancount:1, test_reccnt:37
====event:Gateway, train_len=198, max_len=248, min_len=248
carno:1, totallen:248, nancount:0, test_

====event:Phoenix, train_len=200, max_len=250, min_len=250
carno:1, totallen:250, nancount:0, test_reccnt:48
carno:4, totallen:241, nancount:9, test_reccnt:39
carno:5, totallen:250, nancount:0, test_reccnt:48
carno:6, totallen:250, nancount:0, test_reccnt:48
carno:9, totallen:250, nancount:0, test_reccnt:48
carno:10, totallen:229, nancount:21, test_reccnt:27
a short ts: carid=12，len=154
carno:14, totallen:250, nancount:0, test_reccnt:48
carno:15, totallen:250, nancount:0, test_reccnt:48
carno:18, totallen:249, nancount:1, test_reccnt:47
a short ts: carid=19，len=40
carno:20, totallen:250, nancount:0, test_reccnt:48
carno:21, totallen:249, nancount:1, test_reccnt:47
carno:22, totallen:250, nancount:0, test_reccnt:48
carno:23, totallen:248, nancount:2, test_reccnt:46
carno:26, totallen:249, nancount:1, test_reccnt:47
carno:27, totallen:250, nancount:0, test_reccnt:48
carno:28, totallen:250, nancount:0, test_reccnt:48
carno:30, totallen:250, nancount:0, test_reccnt:48
a short ts: carid=32，

====event:Phoenix, train_len=200, max_len=250, min_len=250
carno:1, totallen:250, nancount:0, test_reccnt:48
carno:4, totallen:241, nancount:9, test_reccnt:39
carno:5, totallen:250, nancount:0, test_reccnt:48
carno:6, totallen:250, nancount:0, test_reccnt:48
carno:9, totallen:250, nancount:0, test_reccnt:48
carno:10, totallen:229, nancount:21, test_reccnt:27
a short ts: carid=12，len=154
carno:14, totallen:250, nancount:0, test_reccnt:48
carno:15, totallen:250, nancount:0, test_reccnt:48
carno:18, totallen:249, nancount:1, test_reccnt:47
a short ts: carid=19，len=40
carno:20, totallen:250, nancount:0, test_reccnt:48
carno:21, totallen:249, nancount:1, test_reccnt:47
carno:22, totallen:250, nancount:0, test_reccnt:48
carno:23, totallen:248, nancount:2, test_reccnt:46
carno:26, totallen:249, nancount:1, test_reccnt:47
carno:27, totallen:250, nancount:0, test_reccnt:48
carno:28, totallen:250, nancount:0, test_reccnt:48
carno:30, totallen:250, nancount:0, test_reccnt:48
a short ts: carid=32，

====event:Phoenix, train_len=200, max_len=250, min_len=250
carno:1, totallen:250, nancount:0, test_reccnt:48
carno:4, totallen:241, nancount:9, test_reccnt:39
carno:5, totallen:250, nancount:0, test_reccnt:48
carno:6, totallen:250, nancount:0, test_reccnt:48
carno:9, totallen:250, nancount:0, test_reccnt:48
carno:10, totallen:229, nancount:21, test_reccnt:27
a short ts: carid=12，len=154
carno:14, totallen:250, nancount:0, test_reccnt:48
carno:15, totallen:250, nancount:0, test_reccnt:48
carno:18, totallen:249, nancount:1, test_reccnt:47
a short ts: carid=19，len=40
carno:20, totallen:250, nancount:0, test_reccnt:48
carno:21, totallen:249, nancount:1, test_reccnt:47
carno:22, totallen:250, nancount:0, test_reccnt:48
carno:23, totallen:248, nancount:2, test_reccnt:46
carno:26, totallen:249, nancount:1, test_reccnt:47
carno:27, totallen:250, nancount:0, test_reccnt:48
carno:28, totallen:250, nancount:0, test_reccnt:48
carno:30, totallen:250, nancount:0, test_reccnt:48
a short ts: carid=32，

...

a short ts: carid=18，len=0
carno:19, totallen:247, nancount:1, test_reccnt:2
carno:20, totallen:247, nancount:1, test_reccnt:2
carno:21, totallen:248, nancount:0, test_reccnt:2
carno:22, totallen:248, nancount:0, test_reccnt:2
carno:23, totallen:235, nancount:13, test_reccnt:1
carno:26, totallen:248, nancount:0, test_reccnt:2
carno:27, totallen:248, nancount:0, test_reccnt:2
a short ts: carid=28，len=173
carno:30, totallen:247, nancount:1, test_reccnt:2
carno:59, totallen:244, nancount:4, test_reccnt:2
carno:88, totallen:242, nancount:6, test_reccnt:1
carno:98, totallen:246, nancount:2, test_reccnt:2
train len:112, test len:179
====event:Phoenix, train_len=200, max_len=250, min_len=250
carno:1, totallen:250, nancount:0, test_reccnt:2
carno:4, totallen:241, nancount:9, test_reccnt:1
carno:5, totallen:250, nancount:0, test_reccnt:2
carno:6, totallen:250, nancount:0, test_reccnt:2
carno:9, totallen:250, nancount:0, test_reccnt:2
a short ts: carid=10，len=229
a short ts: carid=12，len=154
car

====event:Phoenix, train_len=200, max_len=250, min_len=250
carno:1, totallen:250, nancount:0, test_reccnt:2
carno:4, totallen:241, nancount:9, test_reccnt:1
carno:5, totallen:250, nancount:0, test_reccnt:2
carno:6, totallen:250, nancount:0, test_reccnt:2
carno:9, totallen:250, nancount:0, test_reccnt:2
a short ts: carid=10，len=229
a short ts: carid=12，len=154
carno:14, totallen:250, nancount:0, test_reccnt:2
carno:15, totallen:250, nancount:0, test_reccnt:2
carno:18, totallen:249, nancount:1, test_reccnt:2
a short ts: carid=19，len=40
carno:20, totallen:250, nancount:0, test_reccnt:2
carno:21, totallen:249, nancount:1, test_reccnt:2
carno:22, totallen:250, nancount:0, test_reccnt:2
carno:23, totallen:248, nancount:2, test_reccnt:2
carno:26, totallen:249, nancount:1, test_reccnt:2
carno:27, totallen:250, nancount:0, test_reccnt:2
carno:28, totallen:250, nancount:0, test_reccnt:2
carno:30, totallen:250, nancount:0, test_reccnt:2
a short ts: carid=32，len=174
carno:59, totallen:247, nancoun

====event:Texas, train_len=198, max_len=248, min_len=248
carno:1, totallen:244, nancount:4, test_reccnt:2
a short ts: carid=3，len=2
a short ts: carid=4，len=5
carno:5, totallen:248, nancount:0, test_reccnt:2
a short ts: carid=6，len=171
a short ts: carid=7，len=2
carno:9, totallen:248, nancount:0, test_reccnt:2
carno:10, totallen:248, nancount:0, test_reccnt:2
a short ts: carid=12，len=204
a short ts: carid=14，len=31
carno:15, totallen:248, nancount:0, test_reccnt:2
carno:18, totallen:248, nancount:0, test_reccnt:2
a short ts: carid=19，len=205
a short ts: carid=20，len=168
carno:21, totallen:247, nancount:1, test_reccnt:2
carno:22, totallen:248, nancount:0, test_reccnt:2
carno:23, totallen:247, nancount:1, test_reccnt:2
a short ts: carid=25，len=2
carno:26, totallen:238, nancount:10, test_reccnt:1
carno:27, totallen:248, nancount:0, test_reccnt:2
carno:28, totallen:248, nancount:0, test_reccnt:2
carno:30, totallen:248, nancount:0, test_reccnt:2
a short ts: carid=47，len=2
a short ts: carid=55

====event:Phoenix, train_len=200, max_len=250, min_len=250
carno:1, totallen:250, nancount:0, test_reccnt:48
carno:4, totallen:241, nancount:9, test_reccnt:39
carno:5, totallen:250, nancount:0, test_reccnt:48
carno:6, totallen:250, nancount:0, test_reccnt:48
carno:9, totallen:250, nancount:0, test_reccnt:48
carno:10, totallen:229, nancount:21, test_reccnt:27
a short ts: carid=12，len=154
carno:14, totallen:250, nancount:0, test_reccnt:48
carno:15, totallen:250, nancount:0, test_reccnt:48
carno:18, totallen:249, nancount:1, test_reccnt:47
a short ts: carid=19，len=40
carno:20, totallen:250, nancount:0, test_reccnt:48
carno:21, totallen:249, nancount:1, test_reccnt:47
carno:22, totallen:250, nancount:0, test_reccnt:48
carno:23, totallen:248, nancount:2, test_reccnt:46
carno:26, totallen:249, nancount:1, test_reccnt:47
carno:27, totallen:250, nancount:0, test_reccnt:48
carno:28, totallen:250, nancount:0, test_reccnt:48
carno:30, totallen:250, nancount:0, test_reccnt:48
a short ts: carid=32，

====event:Phoenix, train_len=200, max_len=250, min_len=250
carno:1, totallen:250, nancount:0, test_reccnt:8
carno:4, totallen:241, nancount:9, test_reccnt:7
carno:5, totallen:250, nancount:0, test_reccnt:8
carno:6, totallen:250, nancount:0, test_reccnt:8
carno:9, totallen:250, nancount:0, test_reccnt:8
carno:10, totallen:229, nancount:21, test_reccnt:4
a short ts: carid=12，len=154
carno:14, totallen:250, nancount:0, test_reccnt:8
carno:15, totallen:250, nancount:0, test_reccnt:8
carno:18, totallen:249, nancount:1, test_reccnt:8
a short ts: carid=19，len=40
carno:20, totallen:250, nancount:0, test_reccnt:8
carno:21, totallen:249, nancount:1, test_reccnt:8
carno:22, totallen:250, nancount:0, test_reccnt:8
carno:23, totallen:248, nancount:2, test_reccnt:8
carno:26, totallen:249, nancount:1, test_reccnt:8
carno:27, totallen:250, nancount:0, test_reccnt:8
carno:28, totallen:250, nancount:0, test_reccnt:8
carno:30, totallen:250, nancount:0, test_reccnt:8
a short ts: carid=32，len=174
carno:59,

In [9]:
# train_ratio=0.5

freq = "1min"

useeid = False
interpolate = False
#ipstr = '-ip' if interpolate else '-noip'
ipstr = '%s-%s'%('ip' if interpolate else 'noip', 'eid' if useeid else 'noeid')
if useeid:
    cardinality = [len(global_carids), len(laptime_data)]
else:
    cardinality = [len(global_carids)]

plens=[2,5,10,20,30]
for prediction_length in plens:
    make_gluontsdb(COL_TIMEDIFF,'timediff',train_ratio=0.5)
    
#special
interpolate = True
useeid = False
ipstr = '%s-%s'%('ip' if interpolate else 'noip', 'eid' if useeid else 'noeid')
for prediction_length in plens:
    make_gluontsdb(COL_TIMEDIFF,'timediff',train_ratio=0.5)

useeid = True
interpolate = False
#ipstr = '-ip' if interpolate else '-noip'
ipstr = '%s-%s'%('ip' if interpolate else 'noip', 'eid' if useeid else 'noeid')
if useeid:
    cardinality = [len(global_carids), len(laptime_data)]
else:
    cardinality = [len(global_carids)]

for prediction_length in plens:
    make_gluontsdb(COL_TIMEDIFF,'timediff',train_ratio=0.5)

====event:Phoenix, train_len=125, max_len=250, min_len=250
carno:1, totallen:250, nancount:0, test_reccnt:123
carno:4, totallen:241, nancount:9, test_reccnt:114
carno:5, totallen:250, nancount:0, test_reccnt:123
carno:6, totallen:250, nancount:0, test_reccnt:123
carno:9, totallen:250, nancount:0, test_reccnt:123
carno:10, totallen:229, nancount:21, test_reccnt:102
carno:12, totallen:154, nancount:96, test_reccnt:27
carno:14, totallen:250, nancount:0, test_reccnt:123
carno:15, totallen:250, nancount:0, test_reccnt:123
carno:18, totallen:249, nancount:1, test_reccnt:122
a short ts: carid=19，len=40
carno:20, totallen:250, nancount:0, test_reccnt:123
carno:21, totallen:249, nancount:1, test_reccnt:122
carno:22, totallen:250, nancount:0, test_reccnt:123
carno:23, totallen:248, nancount:2, test_reccnt:121
carno:26, totallen:249, nancount:1, test_reccnt:122
carno:27, totallen:250, nancount:0, test_reccnt:123
carno:28, totallen:250, nancount:0, test_reccnt:123
carno:30, totallen:250, nancount:

carno:29, totallen:200, nancount:0, test_reccnt:98
a short ts: carid=30，len=46
carno:32, totallen:110, nancount:90, test_reccnt:8
a short ts: carid=33，len=46
carno:59, totallen:198, nancount:2, test_reccnt:96
carno:60, totallen:200, nancount:0, test_reccnt:98
carno:64, totallen:200, nancount:0, test_reccnt:98
carno:66, totallen:200, nancount:0, test_reccnt:98
carno:88, totallen:200, nancount:0, test_reccnt:98
carno:98, totallen:200, nancount:0, test_reccnt:98
====event:Texas, train_len=124, max_len=248, min_len=248
carno:1, totallen:244, nancount:4, test_reccnt:118
a short ts: carid=3，len=2
a short ts: carid=4，len=5
carno:5, totallen:248, nancount:0, test_reccnt:122
carno:6, totallen:171, nancount:77, test_reccnt:45
a short ts: carid=7，len=2
carno:9, totallen:248, nancount:0, test_reccnt:122
carno:10, totallen:248, nancount:0, test_reccnt:122
carno:12, totallen:204, nancount:44, test_reccnt:78
a short ts: carid=14，len=31
carno:15, totallen:248, nancount:0, test_reccnt:122
carno:18, tot

carno:12, totallen:299, nancount:1, test_reccnt:147
carno:14, totallen:292, nancount:8, test_reccnt:140
carno:15, totallen:299, nancount:1, test_reccnt:147
carno:18, totallen:297, nancount:3, test_reccnt:145
carno:19, totallen:291, nancount:9, test_reccnt:139
carno:20, totallen:298, nancount:2, test_reccnt:146
carno:21, totallen:300, nancount:0, test_reccnt:148
carno:22, totallen:299, nancount:1, test_reccnt:147
carno:23, totallen:295, nancount:5, test_reccnt:143
carno:26, totallen:279, nancount:21, test_reccnt:127
carno:27, totallen:298, nancount:2, test_reccnt:146
carno:28, totallen:283, nancount:17, test_reccnt:131
carno:30, totallen:300, nancount:0, test_reccnt:148
carno:59, totallen:294, nancount:6, test_reccnt:142
a short ts: carid=88，len=100
carno:98, totallen:293, nancount:7, test_reccnt:141
====event:Pocono, train_len=100, max_len=200, min_len=200
carno:1, totallen:199, nancount:1, test_reccnt:97
carno:4, totallen:197, nancount:3, test_reccnt:95
a short ts: carid=5，len=6
a sho

carno:5, totallen:246, nancount:2, test_reccnt:120
carno:9, totallen:248, nancount:0, test_reccnt:122
carno:10, totallen:248, nancount:0, test_reccnt:122
carno:12, totallen:248, nancount:0, test_reccnt:122
carno:14, totallen:246, nancount:2, test_reccnt:120
carno:15, totallen:247, nancount:1, test_reccnt:121
a short ts: carid=18，len=0
carno:19, totallen:247, nancount:1, test_reccnt:121
carno:20, totallen:247, nancount:1, test_reccnt:121
carno:21, totallen:248, nancount:0, test_reccnt:122
carno:22, totallen:248, nancount:0, test_reccnt:122
carno:23, totallen:235, nancount:13, test_reccnt:109
carno:26, totallen:248, nancount:0, test_reccnt:122
carno:27, totallen:248, nancount:0, test_reccnt:122
carno:28, totallen:173, nancount:75, test_reccnt:47
carno:30, totallen:247, nancount:1, test_reccnt:121
carno:59, totallen:244, nancount:4, test_reccnt:118
carno:88, totallen:242, nancount:6, test_reccnt:116
carno:98, totallen:246, nancount:2, test_reccnt:120
train len:126, test len:13830
====even

====event:Phoenix, train_len=125, max_len=250, min_len=250
carno:1, totallen:250, nancount:0, test_reccnt:123
carno:4, totallen:241, nancount:9, test_reccnt:114
carno:5, totallen:250, nancount:0, test_reccnt:123
carno:6, totallen:250, nancount:0, test_reccnt:123
carno:9, totallen:250, nancount:0, test_reccnt:123
carno:10, totallen:229, nancount:21, test_reccnt:102
carno:12, totallen:154, nancount:96, test_reccnt:27
carno:14, totallen:250, nancount:0, test_reccnt:123
carno:15, totallen:250, nancount:0, test_reccnt:123
carno:18, totallen:249, nancount:1, test_reccnt:122
a short ts: carid=19，len=40
carno:20, totallen:250, nancount:0, test_reccnt:123
carno:21, totallen:249, nancount:1, test_reccnt:122
carno:22, totallen:250, nancount:0, test_reccnt:123
carno:23, totallen:248, nancount:2, test_reccnt:121
carno:26, totallen:249, nancount:1, test_reccnt:122
carno:27, totallen:250, nancount:0, test_reccnt:123
carno:28, totallen:250, nancount:0, test_reccnt:123
carno:30, totallen:250, nancount:

...

====event:Phoenix, train_len=125, max_len=250, min_len=250
carno:1, totallen:250, nancount:0, test_reccnt:7
carno:4, totallen:241, nancount:9, test_reccnt:6
carno:5, totallen:250, nancount:0, test_reccnt:7
carno:6, totallen:250, nancount:0, test_reccnt:7
carno:9, totallen:250, nancount:0, test_reccnt:7
carno:10, totallen:229, nancount:21, test_reccnt:5
a short ts: carid=12，len=154
carno:14, totallen:250, nancount:0, test_reccnt:7
carno:15, totallen:250, nancount:0, test_reccnt:7
carno:18, totallen:249, nancount:1, test_reccnt:7
a short ts: carid=19，len=40
carno:20, totallen:250, nancount:0, test_reccnt:7
carno:21, totallen:249, nancount:1, test_reccnt:7
carno:22, totallen:250, nancount:0, test_reccnt:7
carno:23, totallen:248, nancount:2, test_reccnt:7
carno:26, totallen:249, nancount:1, test_reccnt:7
carno:27, totallen:250, nancount:0, test_reccnt:7
carno:28, totallen:250, nancount:0, test_reccnt:7
carno:30, totallen:250, nancount:0, test_reccnt:7
carno:32, totallen:174, nancount:76, t

carno:20, totallen:247, nancount:1, test_reccnt:7
carno:21, totallen:248, nancount:0, test_reccnt:7
carno:22, totallen:248, nancount:0, test_reccnt:7
carno:23, totallen:235, nancount:13, test_reccnt:6
carno:26, totallen:248, nancount:0, test_reccnt:7
carno:27, totallen:248, nancount:0, test_reccnt:7
carno:28, totallen:173, nancount:75, test_reccnt:2
carno:30, totallen:247, nancount:1, test_reccnt:7
carno:59, totallen:244, nancount:4, test_reccnt:6
carno:88, totallen:242, nancount:6, test_reccnt:6
carno:98, totallen:246, nancount:2, test_reccnt:7
train len:124, test len:750
====event:Phoenix, train_len=125, max_len=250, min_len=250
carno:1, totallen:250, nancount:0, test_reccnt:7
carno:4, totallen:241, nancount:9, test_reccnt:6
carno:5, totallen:250, nancount:0, test_reccnt:7
carno:6, totallen:250, nancount:0, test_reccnt:7
carno:9, totallen:250, nancount:0, test_reccnt:7
carno:10, totallen:229, nancount:21, test_reccnt:5
a short ts: carid=12，len=154
carno:14, totallen:250, nancount:0, 

carno:66, totallen:200, nancount:0, test_reccnt:98
carno:88, totallen:200, nancount:0, test_reccnt:98
carno:98, totallen:200, nancount:0, test_reccnt:98
====event:Texas, train_len=124, max_len=248, min_len=248
carno:1, totallen:244, nancount:4, test_reccnt:118
a short ts: carid=3，len=2
a short ts: carid=4，len=5
carno:5, totallen:248, nancount:0, test_reccnt:122
carno:6, totallen:171, nancount:77, test_reccnt:45
a short ts: carid=7，len=2
carno:9, totallen:248, nancount:0, test_reccnt:122
carno:10, totallen:248, nancount:0, test_reccnt:122
carno:12, totallen:204, nancount:44, test_reccnt:78
a short ts: carid=14，len=31
carno:15, totallen:248, nancount:0, test_reccnt:122
carno:18, totallen:248, nancount:0, test_reccnt:122
carno:19, totallen:205, nancount:43, test_reccnt:79
carno:20, totallen:168, nancount:80, test_reccnt:42
carno:21, totallen:247, nancount:1, test_reccnt:121
carno:22, totallen:248, nancount:0, test_reccnt:122
carno:23, totallen:247, nancount:1, test_reccnt:121
a short ts: 

carno:23, totallen:295, nancount:5, test_reccnt:70
carno:26, totallen:279, nancount:21, test_reccnt:62
carno:27, totallen:298, nancount:2, test_reccnt:72
carno:28, totallen:283, nancount:17, test_reccnt:64
carno:30, totallen:300, nancount:0, test_reccnt:73
carno:59, totallen:294, nancount:6, test_reccnt:70
a short ts: carid=88，len=100
carno:98, totallen:293, nancount:7, test_reccnt:69
====event:Pocono, train_len=100, max_len=200, min_len=200
carno:1, totallen:199, nancount:1, test_reccnt:47
carno:4, totallen:197, nancount:3, test_reccnt:46
a short ts: carid=5，len=6
a short ts: carid=6，len=6
carno:9, totallen:200, nancount:0, test_reccnt:48
carno:10, totallen:197, nancount:3, test_reccnt:46
carno:12, totallen:200, nancount:0, test_reccnt:48
a short ts: carid=14，len=16
carno:15, totallen:196, nancount:4, test_reccnt:46
carno:18, totallen:200, nancount:0, test_reccnt:48
a short ts: carid=19，len=6
carno:20, totallen:197, nancount:3, test_reccnt:46
a short ts: carid=21，len=17
carno:22, tota

====event:Phoenix, train_len=125, max_len=250, min_len=250
carno:1, totallen:250, nancount:0, test_reccnt:11
carno:4, totallen:241, nancount:9, test_reccnt:10
carno:5, totallen:250, nancount:0, test_reccnt:11
carno:6, totallen:250, nancount:0, test_reccnt:11
carno:9, totallen:250, nancount:0, test_reccnt:11
carno:10, totallen:229, nancount:21, test_reccnt:9
carno:12, totallen:154, nancount:96, test_reccnt:1
carno:14, totallen:250, nancount:0, test_reccnt:11
carno:15, totallen:250, nancount:0, test_reccnt:11
carno:18, totallen:249, nancount:1, test_reccnt:11
a short ts: carid=19，len=40
carno:20, totallen:250, nancount:0, test_reccnt:11
carno:21, totallen:249, nancount:1, test_reccnt:11
carno:22, totallen:250, nancount:0, test_reccnt:11
carno:23, totallen:248, nancount:2, test_reccnt:11
carno:26, totallen:249, nancount:1, test_reccnt:11
carno:27, totallen:250, nancount:0, test_reccnt:11
carno:28, totallen:250, nancount:0, test_reccnt:11
carno:30, totallen:250, nancount:0, test_reccnt:11


In [19]:
### split by event

freq = "1min"

useeid = False
interpolate = False
#ipstr = '-ip' if interpolate else '-noip'
ipstr = '%s-%s'%('ip' if interpolate else 'noip', 'eid' if useeid else 'noeid')
if useeid:
    cardinality = [len(global_carids), len(laptime_data)]
else:
    cardinality = [len(global_carids)]

plens=[2,5,10,20,30]
for prediction_length in plens:
    make_gluontsdb_byevent(COL_TIMEDIFF,'timediff')
    
#special
interpolate = True
useeid = False
ipstr = '%s-%s'%('ip' if interpolate else 'noip', 'eid' if useeid else 'noeid')
for prediction_length in plens:
    make_gluontsdb_byevent(COL_TIMEDIFF,'timediff')

useeid = True
interpolate = False
#ipstr = '-ip' if interpolate else '-noip'
ipstr = '%s-%s'%('ip' if interpolate else 'noip', 'eid' if useeid else 'noeid')
if useeid:
    cardinality = [len(global_carids), len(laptime_data)]
else:
    cardinality = [len(global_carids)]

for prediction_length in plens:
    make_gluontsdb_byevent(COL_TIMEDIFF,'timediff')

====event:Phoenix, train_len=200, max_len=250, min_len=250
carno:1, totallen:250, nancount:0, test_reccnt:0
carno:4, totallen:241, nancount:9, test_reccnt:0
carno:5, totallen:250, nancount:0, test_reccnt:0
carno:6, totallen:250, nancount:0, test_reccnt:0
carno:9, totallen:250, nancount:0, test_reccnt:0
carno:10, totallen:229, nancount:21, test_reccnt:0
a short ts: carid=12，len=154
carno:14, totallen:250, nancount:0, test_reccnt:0
carno:15, totallen:250, nancount:0, test_reccnt:0
carno:18, totallen:249, nancount:1, test_reccnt:0
a short ts: carid=19，len=40
carno:20, totallen:250, nancount:0, test_reccnt:0
carno:21, totallen:249, nancount:1, test_reccnt:0
carno:22, totallen:250, nancount:0, test_reccnt:0
carno:23, totallen:248, nancount:2, test_reccnt:0
carno:26, totallen:249, nancount:1, test_reccnt:0
carno:27, totallen:250, nancount:0, test_reccnt:0
carno:28, totallen:250, nancount:0, test_reccnt:0
carno:30, totallen:250, nancount:0, test_reccnt:0
a short ts: carid=32，len=174
carno:59,

====event:Phoenix, train_len=200, max_len=250, min_len=250
carno:1, totallen:250, nancount:0, test_reccnt:0
carno:4, totallen:241, nancount:9, test_reccnt:0
carno:5, totallen:250, nancount:0, test_reccnt:0
carno:6, totallen:250, nancount:0, test_reccnt:0
carno:9, totallen:250, nancount:0, test_reccnt:0
carno:10, totallen:229, nancount:21, test_reccnt:0
a short ts: carid=12，len=154
carno:14, totallen:250, nancount:0, test_reccnt:0
carno:15, totallen:250, nancount:0, test_reccnt:0
carno:18, totallen:249, nancount:1, test_reccnt:0
a short ts: carid=19，len=40
carno:20, totallen:250, nancount:0, test_reccnt:0
carno:21, totallen:249, nancount:1, test_reccnt:0
carno:22, totallen:250, nancount:0, test_reccnt:0
carno:23, totallen:248, nancount:2, test_reccnt:0
carno:26, totallen:249, nancount:1, test_reccnt:0
carno:27, totallen:250, nancount:0, test_reccnt:0
carno:28, totallen:250, nancount:0, test_reccnt:0
carno:30, totallen:250, nancount:0, test_reccnt:0
a short ts: carid=32，len=174
carno:59,

====event:Phoenix, train_len=200, max_len=250, min_len=250
carno:1, totallen:250, nancount:0, test_reccnt:0
carno:4, totallen:241, nancount:9, test_reccnt:0
carno:5, totallen:250, nancount:0, test_reccnt:0
carno:6, totallen:250, nancount:0, test_reccnt:0
carno:9, totallen:250, nancount:0, test_reccnt:0
carno:10, totallen:229, nancount:21, test_reccnt:0
a short ts: carid=12，len=154
carno:14, totallen:250, nancount:0, test_reccnt:0
carno:15, totallen:250, nancount:0, test_reccnt:0
carno:18, totallen:249, nancount:1, test_reccnt:0
a short ts: carid=19，len=40
carno:20, totallen:250, nancount:0, test_reccnt:0
carno:21, totallen:249, nancount:1, test_reccnt:0
carno:22, totallen:250, nancount:0, test_reccnt:0
carno:23, totallen:248, nancount:2, test_reccnt:0
carno:26, totallen:249, nancount:1, test_reccnt:0
carno:27, totallen:250, nancount:0, test_reccnt:0
carno:28, totallen:250, nancount:0, test_reccnt:0
carno:30, totallen:250, nancount:0, test_reccnt:0
a short ts: carid=32，len=174
carno:59,

====event:Phoenix, train_len=200, max_len=250, min_len=250
carno:1, totallen:250, nancount:0, test_reccnt:0
carno:4, totallen:241, nancount:9, test_reccnt:0
carno:5, totallen:250, nancount:0, test_reccnt:0
carno:6, totallen:250, nancount:0, test_reccnt:0
carno:9, totallen:250, nancount:0, test_reccnt:0
carno:10, totallen:229, nancount:21, test_reccnt:0
a short ts: carid=12，len=154
carno:14, totallen:250, nancount:0, test_reccnt:0
carno:15, totallen:250, nancount:0, test_reccnt:0
carno:18, totallen:249, nancount:1, test_reccnt:0
a short ts: carid=19，len=40
carno:20, totallen:250, nancount:0, test_reccnt:0
carno:21, totallen:249, nancount:1, test_reccnt:0
carno:22, totallen:250, nancount:0, test_reccnt:0
carno:23, totallen:248, nancount:2, test_reccnt:0
carno:26, totallen:249, nancount:1, test_reccnt:0
carno:27, totallen:250, nancount:0, test_reccnt:0
carno:28, totallen:250, nancount:0, test_reccnt:0
carno:30, totallen:250, nancount:0, test_reccnt:0
a short ts: carid=32，len=174
carno:59,

====event:Phoenix, train_len=200, max_len=250, min_len=250
carno:1, totallen:250, nancount:0, test_reccnt:0
carno:4, totallen:241, nancount:9, test_reccnt:0
carno:5, totallen:250, nancount:0, test_reccnt:0
carno:6, totallen:250, nancount:0, test_reccnt:0
carno:9, totallen:250, nancount:0, test_reccnt:0
carno:10, totallen:229, nancount:21, test_reccnt:0
a short ts: carid=12，len=154
carno:14, totallen:250, nancount:0, test_reccnt:0
carno:15, totallen:250, nancount:0, test_reccnt:0
carno:18, totallen:249, nancount:1, test_reccnt:0
a short ts: carid=19，len=40
carno:20, totallen:250, nancount:0, test_reccnt:0
carno:21, totallen:249, nancount:1, test_reccnt:0
carno:22, totallen:250, nancount:0, test_reccnt:0
carno:23, totallen:248, nancount:2, test_reccnt:0
carno:26, totallen:249, nancount:1, test_reccnt:0
carno:27, totallen:250, nancount:0, test_reccnt:0
carno:28, totallen:250, nancount:0, test_reccnt:0
carno:30, totallen:250, nancount:0, test_reccnt:0
a short ts: carid=32，len=174
carno:59,

...

====event:Phoenix, train_len=200, max_len=250, min_len=250
carno:1, totallen:250, nancount:0, test_reccnt:0
carno:4, totallen:241, nancount:9, test_reccnt:0
carno:5, totallen:250, nancount:0, test_reccnt:0
carno:6, totallen:250, nancount:0, test_reccnt:0
carno:9, totallen:250, nancount:0, test_reccnt:0
carno:10, totallen:229, nancount:21, test_reccnt:0
a short ts: carid=12，len=154
carno:14, totallen:250, nancount:0, test_reccnt:0
carno:15, totallen:250, nancount:0, test_reccnt:0
carno:18, totallen:249, nancount:1, test_reccnt:0
a short ts: carid=19，len=40
carno:20, totallen:250, nancount:0, test_reccnt:0
carno:21, totallen:249, nancount:1, test_reccnt:0
carno:22, totallen:250, nancount:0, test_reccnt:0
carno:23, totallen:248, nancount:2, test_reccnt:0
carno:26, totallen:249, nancount:1, test_reccnt:0
carno:27, totallen:250, nancount:0, test_reccnt:0
carno:28, totallen:250, nancount:0, test_reccnt:0
carno:30, totallen:250, nancount:0, test_reccnt:0
a short ts: carid=32，len=174
carno:59,

====event:Phoenix, train_len=200, max_len=250, min_len=250
carno:1, totallen:250, nancount:0, test_reccnt:0
carno:4, totallen:241, nancount:9, test_reccnt:0
carno:5, totallen:250, nancount:0, test_reccnt:0
carno:6, totallen:250, nancount:0, test_reccnt:0
carno:9, totallen:250, nancount:0, test_reccnt:0
carno:10, totallen:229, nancount:21, test_reccnt:0
a short ts: carid=12，len=154
carno:14, totallen:250, nancount:0, test_reccnt:0
carno:15, totallen:250, nancount:0, test_reccnt:0
carno:18, totallen:249, nancount:1, test_reccnt:0
a short ts: carid=19，len=40
carno:20, totallen:250, nancount:0, test_reccnt:0
carno:21, totallen:249, nancount:1, test_reccnt:0
carno:22, totallen:250, nancount:0, test_reccnt:0
carno:23, totallen:248, nancount:2, test_reccnt:0
carno:26, totallen:249, nancount:1, test_reccnt:0
carno:27, totallen:250, nancount:0, test_reccnt:0
carno:28, totallen:250, nancount:0, test_reccnt:0
carno:30, totallen:250, nancount:0, test_reccnt:0
a short ts: carid=32，len=174
carno:59,

carno:22, totallen:248, nancount:0, test_reccnt:0
carno:23, totallen:235, nancount:13, test_reccnt:0
carno:26, totallen:248, nancount:0, test_reccnt:0
carno:27, totallen:248, nancount:0, test_reccnt:0
a short ts: carid=28，len=173
carno:30, totallen:247, nancount:1, test_reccnt:0
carno:59, totallen:244, nancount:4, test_reccnt:0
carno:88, totallen:242, nancount:6, test_reccnt:0
carno:98, totallen:246, nancount:2, test_reccnt:0
train len:88, test len:191
====event:Phoenix, train_len=200, max_len=250, min_len=250
carno:1, totallen:250, nancount:0, test_reccnt:0
carno:4, totallen:241, nancount:9, test_reccnt:0
carno:5, totallen:250, nancount:0, test_reccnt:0
carno:6, totallen:250, nancount:0, test_reccnt:0
carno:9, totallen:250, nancount:0, test_reccnt:0
a short ts: carid=10，len=229
a short ts: carid=12，len=154
carno:14, totallen:250, nancount:0, test_reccnt:0
carno:15, totallen:250, nancount:0, test_reccnt:0
carno:18, totallen:249, nancount:1, test_reccnt:0
a short ts: carid=19，len=40
car

====event:Phoenix, train_len=200, max_len=250, min_len=250
carno:1, totallen:250, nancount:0, test_reccnt:0
carno:4, totallen:241, nancount:9, test_reccnt:0
carno:5, totallen:250, nancount:0, test_reccnt:0
carno:6, totallen:250, nancount:0, test_reccnt:0
carno:9, totallen:250, nancount:0, test_reccnt:0
a short ts: carid=10，len=229
a short ts: carid=12，len=154
carno:14, totallen:250, nancount:0, test_reccnt:0
carno:15, totallen:250, nancount:0, test_reccnt:0
carno:18, totallen:249, nancount:1, test_reccnt:0
a short ts: carid=19，len=40
carno:20, totallen:250, nancount:0, test_reccnt:0
carno:21, totallen:249, nancount:1, test_reccnt:0
carno:22, totallen:250, nancount:0, test_reccnt:0
carno:23, totallen:248, nancount:2, test_reccnt:0
carno:26, totallen:249, nancount:1, test_reccnt:0
carno:27, totallen:250, nancount:0, test_reccnt:0
carno:28, totallen:250, nancount:0, test_reccnt:0
carno:30, totallen:250, nancount:0, test_reccnt:0
a short ts: carid=32，len=174
carno:59, totallen:247, nancoun

====event:Phoenix, train_len=200, max_len=250, min_len=250
carno:1, totallen:250, nancount:0, test_reccnt:0
carno:4, totallen:241, nancount:9, test_reccnt:0
carno:5, totallen:250, nancount:0, test_reccnt:0
carno:6, totallen:250, nancount:0, test_reccnt:0
carno:9, totallen:250, nancount:0, test_reccnt:0
carno:10, totallen:229, nancount:21, test_reccnt:0
a short ts: carid=12，len=154
carno:14, totallen:250, nancount:0, test_reccnt:0
carno:15, totallen:250, nancount:0, test_reccnt:0
carno:18, totallen:249, nancount:1, test_reccnt:0
a short ts: carid=19，len=40
carno:20, totallen:250, nancount:0, test_reccnt:0
carno:21, totallen:249, nancount:1, test_reccnt:0
carno:22, totallen:250, nancount:0, test_reccnt:0
carno:23, totallen:248, nancount:2, test_reccnt:0
carno:26, totallen:249, nancount:1, test_reccnt:0
carno:27, totallen:250, nancount:0, test_reccnt:0
carno:28, totallen:250, nancount:0, test_reccnt:0
carno:30, totallen:250, nancount:0, test_reccnt:0
a short ts: carid=32，len=174
carno:59,

### test

In [42]:
def test_a(a, bitflag):
    return (a & bitflag) ==  bitflag

test_a(5, 4)


True

In [45]:
test_a(3, 2)

True